# Problem statement
You have to write a python script which can fetch all the tweets(as many as allowed by Twitter API) done by midas@IIITD twitter handle and dump the responses into JSONlines file. The other part of your script should be able to parse these JSONline files to display the following for every tweet in a tabular format.

The text of the tweet.
Date and time of the tweet.
The number of favorites/likes.
The number of retweets.
Number of Images present in Tweet. If no image returns None.

In [3]:
import tweepy
from tweepy import OAuthHandler
import json
import os
import pandas as pd

In [5]:
FILE_PATH = r'../Python_problem/tweets.json'

In [6]:
ACCESS_TOKEN = "#"
ACCESS_TOKEN_SECRET = "#"
CONSUMER_KEY = "#"
CONSUMER_SECRET_KEY = "#"

In [7]:
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [8]:

def fetch_and_dump_tweets(screen_name):
    '''
    Parameters-
        screen_name: Twitter handle of the user
    Variables -
        tweets: List (of length upto 200) of tweets by screen_name
        oldest: Index of oldest fetched tweet
        all_tweets_json: List of _json data from all_tweets
    Returns -   
        all_tweets: List of all tweets by screen_name
    '''
    #all_tweets intialized as an empty list
    all_tweets = []
    
    #most recent tweets upto maximum of 200
    tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #add tweets to all_tweets
    all_tweets.extend(tweets)
    
    #ID of the most recent fetched tweet - 1
    oldest = all_tweets[-1].id - 1
    
    #fetch tweets that are older than most recent 200 tweets until no tweet is left or 3200 limit is reached
    while len(tweets) > 0:
        
        tweets = api.user_timeline(screen_name = screen_name, count=200, max_id=oldest)
        all_tweets.extend(tweets)
        oldest = all_tweets[-1].id - 1

    print(f"Total number of tweets from {screen_name} are {len(all_tweets)}")
    
    #all_tweets_json initialized as an empty list
    all_tweets_json = []
    
    #append _json corresponding to each of the tweet to the all_tweets_json
    for tweet in all_tweets:
        all_tweets_json.append(tweet._json)
        
    #dumping all_tweets_json to the file specified by FILE_PATH
    #sort_keys is set to True for sorting dictionaries by key
    #indent = 4, for pretty printing JSON array elements with indent level of 4
    with open(FILE_PATH, 'w', encoding='utf8') as f:
        json.dump(all_tweets_json, f, sort_keys = True,indent = 4)
    return all_tweets

In [ ]:
tweets_by_MIDAS = fetch_and_dump_tweets("midasIIITD")